In [1]:
#### load packages, *.h5 folder and exp.h5 file location
%matplotlib widget

from essential_func import *
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

samples_csv                = 'data_directory.csv'
default_sample_dir         = 'July-2021-Sample#6'
csv_and_code_abs_directory = '/Users/bashit.a/Documents/Alzheimer/Codes/'

dropdown_name_list, cwd, exp_folder = change_python_path(dropdown_name = default_sample_dir, csv_file_location = csv_and_code_abs_directory, samples_csv = samples_csv)

Python directory is set to load samples from : July-2021-Sample#6


In [4]:
### show heat map for any file -- (Independent cell) - files are required to have azimuthal averaged
%matplotlib widget
from ipywidgets import fixed

scatterings = ('_SAXS', '_WAXS2',)  # ('_SAXS', '_WAXS2')
default_saxs_inc = 1
default_waxs_inc = 0.001

dropdown_dir = ipywidgets.Dropdown(options= dropdown_name_list, value = default_sample_dir, description='File ', disabled=False)
dropdown_file = ipywidgets.Dropdown(options= cwd_files_search_with('.h5'), value = cwd_files_search_with('.h5')[0], description='File ', disabled=False)
cmap = ipywidgets.Dropdown(options= cmap_list(), value = 'jet', description='Color Map ', disabled=False)
data_binning = ipywidgets.Checkbox(value=True, description='Data Binning ', disabled=False, style= {'description_width': 'initial'})
saxs_max = ipywidgets.FloatSlider(value = 10,  min= 0,      max=500, step=1 ,   description='SAXS Max:', continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    
saxs_inc = ipywidgets.FloatSlider(value = default_saxs_inc, min= 0.0001, max=saxs_max.value, step=0.001 , description='SAXS Inc.:', continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    
waxs_max = ipywidgets.FloatSlider(value = 0.035,  min= 0,   max=0.045, step=0.001 , description='WAXS Max:', continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    
waxs_inc = ipywidgets.FloatSlider(value = default_waxs_inc, min= 0.0001, max=waxs_max.value, step=0.001 , description='WAXS Inc.:', continuous_update=False, layout=ipywidgets.Layout(width='25%'), readout=True, readout_format='.4f', style= {'description_width': 'initial'})    

### change dropdown_file lists when directory is changed
def update_samples_list(*args):
    global cwd, exp_folder
    _, cwd, exp_folder = change_python_path(dropdown_name = dropdown_dir.value, csv_file_location = csv_and_code_abs_directory, samples_csv = samples_csv)
    dropdown_file.options = cwd_files_search_with('.h5')
    dropdown_file.value = cwd_files_search_with('.h5')[0]
dropdown_dir.observe(update_samples_list)

def get_saxs_waxs_max(*args):
    
    file = dropdown_file.value
    for scattering in scatterings:
        
        Iq = read_Iq(file, scattering)      # Iq shape (3721, 690)
        diff_patterns = find_rep_value(qgrid2, Iq, method = 'circ')
                
        if scattering == '_SAXS':
            saxs_max.max = np.max(diff_patterns)
            saxs_max.value = np.percentile(diff_patterns,90)  # maximum increment is 90 percentile of heat SAXS map
            saxs_inc.value = default_saxs_inc
            saxs_inc.max  = saxs_max.value  
            saxs_inc.step = saxs_inc.max/1000
        
        elif scattering == '_WAXS2':
            waxs_max.max = np.max(diff_patterns)
            waxs_max.value = np.percentile(diff_patterns,99)  # maximum increment is 99 percentile of heat WAXS map
            waxs_inc.value = default_waxs_inc
            waxs_inc.max   = waxs_max.value
            waxs_inc.step = waxs_inc.max/1000
            
get_saxs_waxs_max()    # execute to set vales in accordance with default dropdown setting
dropdown_file.observe(get_saxs_waxs_max, names='value', type='change')  # types and value is default

def heat_map(file, cmap, data_binning, saxs_max, saxs_inc, waxs_max, waxs_inc):
    
    #plot_one_heat_maps(file, scatterings)
    f = plt.figure(figsize=(17,6), constrained_layout=False)
    ax = f.subplots(nrows=1, ncols=2, )
    f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings, heatmap_rep_value = 'circ', arg_qvalue = None, cmap = cmap, args = (f,ax), data_binning=data_binning, bins = [np.fromiter(drange(0, saxs_max, saxs_inc), float), np.fromiter(drange(0, waxs_max, waxs_inc), float)] )  # discrete_cmap(N=5, base_cmap = 'cubehelix')     viridis
    print(f'File = {file}, SAXS-MAX = {saxs_max:0.3f}, SAXS-INC = {saxs_inc:0.3f}, WAXS-MAX = {waxs_max:0.3f}, WAXS-INC = {waxs_inc:0.3f}, cmap = {cmap}')
    plt.tight_layout()

ui  = ipywidgets.VBox([ipywidgets.HBox([dropdown_dir, dropdown_file, cmap, data_binning]), ipywidgets.HBox([saxs_max, saxs_inc, waxs_max, waxs_inc ])])
out = ipywidgets.interactive_output(heat_map, {'file' : dropdown_file, 'cmap' : cmap, 'data_binning' : data_binning, 'saxs_max' : saxs_max, 'saxs_inc' : saxs_inc , 'waxs_max' : waxs_max, 'waxs_inc' : waxs_inc})
display(out, ui)

Output()

In [13]:
### Heatmap at particular q value / point fitting  --- check for point heatmap
%matplotlib widget
file = '2428_FC-roi0.h5'
scatterings = ('_WAXS2',)  # ('_SAXS', '_WAXS2')

f = plt.figure(figsize=(17,6), constrained_layout=False)
ax = f.subplots(nrows=1, ncols=1, )
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings, heatmap_rep_value = 'point', arg_qvalue = 1.40, cmap = 'jet', args = (f,ax), data_binning=False, bins = [np.fromiter(drange(0, saxs_max.value, saxs_inc.value), float), np.fromiter(drange(0, waxs_max.value, waxs_inc.value), float)] )  # discrete_cmap(N=5, base_cmap = 'cubehelix')     viridis

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

qvalue is : 1.4


In [5]:
### load pixalated sum either directly from h5 file or already stored pixalted sum npz file -------- ##### (Independent Cell) - loading from file

# spec
file = '2428_FC-roi0.h5'
load_from = 'npz' # 'h5'

### semi spec
folder = 'pixalated_sum_waxs'
if load_from = 'h5':
    ##### Load file before WAXS thresholding (Requires to run the folliwing cell only) - a 300x300 roi should not take more than 3 min to load for M1 macbook 
    dset_waxs_sum = pixalated_sum_waxs(file, save_as_file=False)

elif load_from = 'npz':
    ### load pixalated sum file (2048_B8.h5-pixalated_sum_waxs.npz) from the saved folder (pixalated_sum_waxs)
    try:
        arr = np.load(f'{folder}/{file.strip(".h5")}-pixalated_sum_waxs.npz', allow_pickle=True)
        dset_waxs_sum      = arr['waxs_sum']
        dset_waxs_sum_stat = arr['waxs_sum_stat']
        dset_waxs_sum_df   = arr['waxs_sum_percentile']
        print(dset_waxs_sum_stat)
        print(dset_waxs_sum_df)
    except:
        print(f'{folder}/{file}-pixalated_sum_waxs.npz file not found')
else:
    print('Something Went Wrong')

['Pixalated Sum WAXS shape = ' (1043, 981) 'Min. = -7812.000000'
 'Mean = 15582.118657' 'Median = 9496.000000' 'Max. = 205089751.000000']
[[70 '11,287.00']
 [71 '11,358.00']
 [72 '11,428.00']
 [73 '11,502.00']
 [74 '11,579.00']
 [75 '11,660.00']
 [76 '11,743.00']
 [77 '11,830.14']
 [78 '11,919.00']
 [79 '12,008.00']
 [80 '12,101.00']
 [81 '12,200.00']
 [82 '12,307.00']
 [83 '12,422.00']
 [84 '12,550.00']
 [85 '12,689.00']
 [86 '12,851.00']
 [87 '13,024.00']
 [88 '13,193.00']
 [89 '13,353.00']
 [90 '13,501.00']
 [91 '13,639.00']
 [92 '13,773.00']
 [93 '13,904.00']
 [94 '14,042.08']
 [95 '14,214.00']
 [96 '14,493.00']
 [97 '15,239.00']
 [98 '17,237.00']
 [99 '28,595.36']]


In [6]:
## WAXS thresholding and loading ------------ (Dependent Cell - requires running either of the previou cells deping on loading pixalted sum on RAM or loading from file)
## you don't need azimuthal averaging of the original file to do masking
## specs/Slider Properties
AMIN_DEFAULT_VALUE = 0       # minimum value for the pixalated sum
AMAX_DEFAULT_VALUE = np.median(dset_waxs_sum) ; AMAX_MIN_VALUE = dset_waxs_sum.min() ; AMAX_MAX_VALUE = np.percentile(dset_waxs_sum, 99.9);  AMAX_SLIDER_STEP = 100;    # maximum values for the pixalated sum
# valid_range_min, valid_range_max = (0,10)   # just for WAXS Display purpose

n_patterns = list(map(lambda x,y: x*y, *zip(width_height(file))))[0]   # calculate number of frames


def waxs_thresholding(a_min, a_max, thr, frame):
    %matplotlib widget
    
    # load waxs frame data 
    with h5py.File(file,'r') as hdf:
        dset_waxs = hdf.get(f'{h5_top_group(file)}/primary/data/pilW2_image')         # waxs data read from h5 file
        dset_waxs = np.array(dset_waxs[frame])
        
    args = (dset_waxs_sum, a_min, a_max, thr)
    dset_waxs_thr, gray_img, thr_fr_img = threshold_patch_one_frame(dset_waxs, args = args)
    
    #### plot thresholded pixalated sum
    f, axs = plt.subplots(nrows=2,ncols=2,figsize=(12,8),num=f'{file}')
    im = axs[0,0].imshow(dset_waxs_thr, cmap = 'rainbow')       #plt.colorbar(im, fraction=0.046, pad=0.04)  
    show_colorbar(im,f,axs[0,0])
    axs[0,0].set_title('Pixelated Sum')
    #### plot global thresholding  
    im = axs[0,1].imshow(gray_img, cmap='gray')       
    axs[0,1].set_title(f'Binary Threshold = {thr}')

    patching(file, frame, qgrid2, args=args, axes = (f,[axs[1,0], axs[1,1]]), method = 'thresholding', exp_folder = exp_folder  )

    print('a_min = ', a_min, 'a_max = {:0.2f}'.format(a_max), 'thr = ', thr)

frame = ipywidgets.IntSlider(value=0, min=0, max=n_patterns-1, step=1, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
a_min = ipywidgets.fixed(value=AMIN_DEFAULT_VALUE)
a_max = ipywidgets.FloatSlider(value=AMAX_DEFAULT_VALUE, min=AMAX_MIN_VALUE,     max=AMAX_MAX_VALUE, step=AMAX_SLIDER_STEP, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
thr   = ipywidgets.IntSlider(value=a_min.value,          min=AMIN_DEFAULT_VALUE, max=AMAX_MAX_VALUE, step=AMAX_SLIDER_STEP, continuous_update=False, layout=ipywidgets.Layout(width='50%'))
ipywidgets.jslink((a_max,'value'),(thr,'max'))           # make sure threshold's maximum value doesn't exceed a_max value

ipywidgets.interact(waxs_thresholding, a_min=a_min, a_max=a_max, thr=thr, frame=frame)

interactive(children=(FloatSlider(value=9496.0, continuous_update=False, description='a_max', layout=Layout(wi…

<function __main__.waxs_thresholding(a_min, a_max, thr, frame)>

In [78]:
##### --------- 1-d averaging after thresholding for one file  --------- (Independent Cell)
### spec
source_file = '2346_Dentate_gyrus-roi1.h5'
args =  0, 19888, 17000   # manually can set args = (a_min.value, a_max.value, thr.value) = 0, 30000, 16000 or Get values from 3rd cell print output

### computation
masked_file = circ_avg_from_patches(source_file, qgrid2, args=args, method = 'thresholding', exp_folder = exp_folder  )

2346_Dentate_gyrus-roi1_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-6 ...
2346_Dentate_gyrus-roi1_masked.h5 copy is done
2346_Dentate_gyrus-roi1_masked.h5 Loading data into a numpy array started 
2346_Dentate_gyrus-roi1_masked.h5 Loading data into a numpy array finished in 44.771737875000326 seconds
2346_Dentate_gyrus-roi1_masked.h5 Patching Started 
2346_Dentate_gyrus-roi1_masked.h5 Patching finished in 45.28245754100135 seconds
2346_Dentate_gyrus-roi1_masked.h5 patched pilW2_image dataset creation staring ... 
2346_Dentate_gyrus-roi1_masked.h5 patched pilW2_image dataset creation finished in 99.11726783300037 seconds
Circular averaging starts now ... 
data received: sn=2346_Dentate_gyrus-roi1, fr1=0
data received: sn=2346_Dentate_gyrus-roi1, fr1=488
data received: sn=2346_Dentate_gyrus-roi1, fr1=976
data received: sn=2346_Dentate_gyrus-roi1, fr1=1464
data received: sn=2346_Dentate_gyrus-roi1, fr1=1952
data received: sn=2346_Dentate_gyru

OSError: Can't write data (file write failed: time = Fri Aug 13 22:14:50 2021
, filename = '2346_Dentate_gyrus-roi1_masked.h5', file descriptor = 73, errno = 28, error message = 'No space left on device', buf = 0x3697f0000, total write size = 43122240, bytes this sub-write = 43122240, bytes actually written = 18446744073709551615, offset = 0)

In [35]:
############### -------- configure patch size and copy the pathces tuplte to the next cell for computing 1-d averaging for all frames ------------- ################### (Independent Cell)
%matplotlib widget
frame = 1326         # possible_bkg frame
file = '2048_B8_masked.h5'
patches = patches = [
#                              [[800, 0], [960, 120]],
#                              [[0, 0], [960, 120]],
#                              [[590, 220], [610, 240]],
#                              [[0, 590], [45, 615]],
#                              [[315, 740], [345, 765]],
#                              [[630, 900], [655, 920]],
#                              [[20, 955], [70, 985]],
#                              [[890, 375], [920, 400]],
#                              [[865, 990], [985, 1030]],
#                              [[310, 420], [330, 435]]
                                [[380,970],[430,1030]]
                    ]

patching(file, frame, qgrid2, args=patches, method = 'rec_circ_patch');

frame information extraction completes with _SAXS shape (1, 1043, 981) _WAXS shape (1, 1043, 981)...
Circular averaging starts now ... 
2048_B8_masked_1326.h5 total 1-d averaging time 0.45188058298663236 seconds


/Users/bashit.a/Documents/Alzheimer/Codes/py4xs/data2d.py:350: RuntimeWarning: invalid value encountered in sqrt
  dI = np.sqrt(Iq2-Iq*Iq)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# 1-d averaging after rectangular and/or circular patches for one file  (Independent Cell)
### spec
source_file = "2048_B8.h5"
patches = [
#                              [[800, 0], [980, 150]],
#                              [[245, 0], [385, 130]],
          ]

### computation
masked_file = circ_avg_from_patches(source_file, qgrid2, patches)

DID not PATCH 2017_EC-roi3_masked.h5 already exists - did not copy, 
1-D Averaging is NOT performed as 2017_EC-roi3_masked.h5 already exist 
TO do patching on 2017_EC-roi3_masked.h5 delete the file first manually and run this cell again


In [2]:
## patching information retrival from _masked.h5 file    (Independent Cell)
#specs
file = '2346_EC-roi0_masked.h5'
method = 'thresholding'

# computation
try: patches = get_patch_attributes(f'{file}', method = 'thresholding') if method == 'thresholding' else get_patch_attributes(f'{file}'); print(patches)
except: print(f'{file} does not have patches information on the processed folder')

2346_EC-roi0_masked.h5 does not have patches information on the processed folder


In [20]:
### check config.json file integrity     (Independent Cell) - no error no problem
No_error_no_prb = get_json_str_data('sample-6-july-2021-withthreshold.json');

In [3]:
# 1-d averaging after patches for ALL files in the current directory   (Independent Cell)

# specs
data = get_json_str_data("config-mar-24-withthreshold.json")
method = 'thresholding'                                        # 'thresholding' or 'rec_circ_patch'

### do masking and 1-d averaging (Lin Yang's code by BNL used here for 1-d averaging)
files = []

for file in data['files']:
    source_file = file['name']
    try: 
        if method == 'rec_circ_patch':
            patches = file['patches']
            masked_file = circ_avg_from_patches(source_file, qgrid2, patches) 
        elif method == 'thresholding': 
            patches = file['threshold']
            print(file, 'thresholded to -->', patches)
            masked_file = circ_avg_from_patches(source_file, qgrid2, args=tuple(patches), method = 'thresholding')  # args must be of tuple
    except: continue
    files.append((source_file, masked_file))

print(f'Successfully created (source file, masked file) \n\n')
[print(idx+1, '. ', s, '--->' ,t) for idx, (s,t) in enumerate(files)]
pass

{'name': '1886_EC-roi0.h5', 'possible_plaque': None, 'possible_bkg': 424, 'patches': [], 'threshold': [0, 44393, 15210], 'comments': ''} thresholded to --> [0, 44393, 15210]
1886_EC-roi0_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Mar-24/BNL-Data ...
1886_EC-roi0_masked.h5 copy is done
1886_EC-roi0_masked.h5 Loading data into a numpy array started 
1886_EC-roi0_masked.h5 Loading data into a numpy array finished in 55.423180167 seconds
1886_EC-roi0_masked.h5 Patching Started 
1886_EC-roi0_masked.h5 Patching finished in 46.708706666 seconds
1886_EC-roi0_masked.h5 patched pilW2_image dataset creation staring ... 
1886_EC-roi0_masked.h5 patched pilW2_image dataset creation finished in 89.06730066699998 seconds
Circular averaging starts now ... 
data received: sn=1886_EC-roi0, fr1=0
data received: sn=1886_EC-roi0, fr1=480
data received: sn=1886_EC-roi0, fr1=960
data received: sn=1886_EC-roi0, fr1=1440
data received: sn=1886_EC-roi0, fr1=1920
data received: sn=1886_EC-r

In [11]:
### 1d circularly averaged data for masked file
%matplotlib widget
import ipywidgets   # from IPython.display import display; display(amp)

# specs 
source_file = '2029_Dentate_gyrus-roi0_masked.h5'          # must input source file name
# masked_file = "2109_EC-roi1.h5"          # comment out to avoid plotting
scattering = ('_SAXS', '_WAXS2',)

# computation
# anyinitialized masked_file will be removed beginning of the cell
try : del masked_file
except NameError: pass

Width, Height = width_height(source_file)    # considering the fact that width and height is same for source and masked file
n_patterns = Width*Height

f,axs = plt.subplots(nrows=2, ncols=3, figsize=(16,8))
# plot_heat_map_from_file(source_file, qgrid2, scatterings = (scattering[1],), cmap = 'viridis', args = (f, axs[0,1]))
plot_heat_map_from_file(source_file, qgrid2, scatterings = (scattering[1],) , cmap = discrete_cmap(N=4, base_cmap = 'jet'), args = (f, axs[0,2]), data_binning=True, bins = np.fromiter(drange(0, 0.04, 0.001), float) )  # discrete_cmap(N=5, base_cmap = 'cubehelix')     viridis
plot_heat_map_from_file(source_file, qgrid2, scatterings = (scattering[0],) , cmap = 'jet', args = (f, axs[1,2]), data_binning=False, bins = np.fromiter(drange(0, 20, 0.5), float) )  # discrete_cmap(N=5, base_cmap = 'cubehelix')     viridis

def update_plot(frame):
    axs[0,0].clear()
    axs[1,0].clear()
    
    axs[0,0].set_yscale('log')
    axs[1,0].set_yscale('log')
    
    axs[0,0].set_xlabel('q'); axs[0,0].set_ylabel('log(I)');
    axs[1,0].set_xlabel('q'); axs[1,0].set_ylabel('log(I)');
    
    Iq_S_SAXS = read_Iq(source_file, scattering[0])   # Source must be read even you do mask or not
    Iq_S_WAXS = read_Iq(source_file, scattering[1])   # Source must be read even you do mask or not

    try: 
        masked_file
        if source_file ==  masked_file:
            axs[0,0].plot(qgrid2, Iq_S_WAXS[frame], label = f' Fr- {frame}');
            waxs_diff_image(source_file, frame, f, axs[0,1])

            axs[1,0].plot(qgrid2, Iq_S_SAXS[frame], label = f' Fr- {frame}');
            saxs_diff_image(source_file, frame, f, axs[1,1])

        else:
            Iq_M = read_Iq(masked_file, scattering)
            axs[0,0].plot(qgrid2, Iq_S[frame], label = f'Before Mask Fr- {frame}'); 
            axs[0,0].plot(qgrid2, Iq_M[frame], label = f'After Mask Fr- {frame}');
            waxs_diff_image(file, frame, f, axs[0,1])


    except NameError: 
        axs[0,0].plot(qgrid2, Iq_S_WAXS[frame], label = f' Fr- {frame}');
        waxs_diff_image(source_file, frame, f, axs[0,1])

        axs[1,0].plot(qgrid2, Iq_S_SAXS[frame], label = f' Fr- {frame}');
        saxs_diff_image(source_file, frame, f, axs[1,1])

    else: print('Something Wrong!')

    axs[0,0].legend()
    axs[1,0].legend()
    plt.suptitle(f'{source_file}')
    plt.tight_layout()

frame = ipywidgets.IntSlider(min=0, max=n_patterns-1, value=0, description = "Frame")
ipywidgets.interactive(update_plot, frame=frame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Frame', max=7811), Output()), _dom_classes=('widget-inte…

In [126]:
%matplotlib widget
file = '2109_EC-roi0.h5'
scattering = '_SAXS'
Iq = read_Iq(file, scattering)      # Iq shape (3721, 690)
diff_patterns = find_rep_value(qgrid2, Iq, method = 'circ')

print(diff_patterns.max(), diff_patterns.min(), diff_patterns.shape)
bins = np.array([0,10,20, 30])
diff_patterns = np.digitize(diff_patterns, bins, right=False)
print(len(inds))

Width, Height = width_height(file)
img_orig = snaking(Width, Height, diff_patterns)
f, axs = plt.subplots(1, len(scatterings), num=f'{file} Heat maps', figsize=(10,5))
plot_heat_map_from_data(img_orig, Width, Height, args = (f, axs[0]), title= f'{scattering} {file}', cmap='Accent')

low valid idx = 0, low valid Q = 0.005, high valid idx = 115 , high valid Q = 0.325
58.311851377490186 0.18907160599802383 (7564,)
7564


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
################# --------- Find Q range to plot heat maps --------- #################
%matplotlib widget
import ipywidgets         # from IPython.display import display; display(amp)
from ipywidgets import fixed

# semi-specs
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

### q_low, q_high, q_low - idx[2] and q_high + idx[2] for poly fit, number of points
indices = ((0.115, 0.19, 13,   48, '_SAXS' , 'bumpy'),      # 72, 81
           (1.48,  1.57, 10, 4.15, '_WAXS2', 'paraffin'),   # 345, 355
           (1.30,  1.42, 10,  4.7, '_WAXS2', 'amyloid'),    # 309, 323
           (1.89,  1.93,  7,  3.2, '_WAXS2', 'mica')        # 427, 433
          )

## polyhit with heatmap from drop down list
files = cwd_files_search_with('.h5')
dropdown = ipywidgets.Dropdown(options=files, value = files[0],description='File : ', disabled=False)
ipywidgets.interactive(file_polyfit_heatmap_plot, file = dropdown, indices = fixed(indices), qgrid2 = fixed(qgrid2))

## polyhit with heatmap for one file
# file = '2048_B8.h5'
# file_polyfit_heatmap_plot(file, indices, qgrid2)

interactive(children=(Dropdown(description='File : ', options=('2029_Dentate_gyrus-roi0.h5', '2029_Dentate_gyr…

In [2]:
### Merge all PDFs in the directory specified
### you can put all your pdfs that you want to merge to the PDF folder
from essential_func import *
pdfs_merging(directory = '/PDF', output = 'result.pdf')

Current location /Users/bashit.a/Documents/Alzheimer/Dec-15
Changing to /Users/bashit.a/Documents/Alzheimer/Dec-15//PDF directory 
result.pdf file written successfully
Back to root directory  /Users/bashit.a/Documents/Alzheimer/Dec-15


In [59]:
%matplotlib widget
from essential_func import *
from py4xs.hdf import h5xs
from py4xs.slnxs import trans_mode

print(f'{os.getcwd()}')

# specs
# file = '1934_SN_masked.h5'  
# frames = (4518, 3687, 3045)     # amyloid, mica, dark  ---> frames can be choosen from SAXS heat map

file = '2346_EC-roi0.h5'
frames = (2080, 1358, 2223, )     # amyloid, mica, dark
show_eb = False                  # show errorbar in the plot

## semi-spec
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

# creating objects
dt  = h5xs(f'{file}', transField="em2_sum_all_mean_value")
sn = h5_top_group(file)
dt.load_d1s(sn=sn)

## heat map from file
scatterings = ('_SAXS', '_WAXS2')
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings)

### 1-d plot for bright, moderate and dark frames
scatterings = ('_SAXS',          '_WAXS2',        'merged',    )
colors =      (('r','b','g'), ('r','b','g'), ('r+','bo','g--'),)
f, axs = plt.subplots(1, 3, figsize=(15,5), num=f'{file}')
xscale = 'linear'
yscale = 'log'

for i, (scattering, color) in enumerate(zip(scatterings,colors)):
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[0]].data, color[0], label = f'bright {frames[0]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[1]].data, color[1], label = f'moderate {frames[1]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[2]].data, color[2], label = f'dark {frames[2]}')
    axs[i].set(title = f'{scattering} {file}', xscale= xscale, yscale= yscale)
    axs[i].legend()

## background correction calculations
sc_factor=1.

dset = dt.d1s[sn][scattering][frames[0]]
dbak = dt.d1s[sn][scattering][frames[2]]

dt.set_trans(transMode=trans_mode.external)
sc  = dset.trans / dbak.trans
print(f'Scaling factor, {dset.trans}/{dbak.trans} = {sc}')
Iq  = dset.data - dbak.data * sc * sc_factor
dI  = dset.err  + dbak.err * sc * sc_factor
idx = (dset.data > 0) & (dbak.data > 0)

## plotting settings
f, axs = plt.subplots(1,2,figsize=(16,4))
plt.subplots_adjust(bottom=0.15)

axs[0].plot(dbak.qgrid[idx], dset.data[idx], color[0], label = f'bright {frames[0]}')
axs[0].plot(dbak.qgrid[idx], dbak.data[idx], color[2], label = f'dark-bcg {frames[2]}')

if show_eb:
    axs[0].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx])
    axs[1].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx], label = f'{frames[0]} bcor')
else:
    axs[0].plot(dbak.qgrid[idx], Iq[idx], label = 'bcor')
    axs[1].plot(dbak.qgrid[idx], Iq[idx], label = f'{frames[0]} bcor')


axs[0].set(xlabel = "$q (\AA^{-1})$", ylabel = "$I$", title = f'Background correction {file} errorbar = {show_eb}', xscale = xscale, yscale = yscale, )
axs[1].set(xscale = xscale, yscale = yscale, title = f'Final Background Subtracted {file} errorbar = {show_eb}' , )
axs[0].legend()
axs[1].legend()
print(Iq[1:10])

/Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 0, low valid Q = 0.005, high valid idx = 115 , high valid Q = 0.325
low valid idx = 87, low valid Q = 0.185, high valid idx = 619 , high valid Q = 2.845


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Scaling factor, 42707.56799999999/42645.287999999986 = 1.0014604192613263


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-55.22141063 -30.60641477 -17.68525177 -11.83066418  -7.53639736
  -5.28041574  -3.98487398  -2.80026383  -2.07096635]


In [3]:
### learn this part of the code
def flatten(x):
    """
    l = [2,[[1,2]],1]
    list(flatten(l))
    """
    for item in x:
        if type(item) in [list]:
            for num in flatten(item):
                yield(num)
        else:
            yield(item)

flatten=lambda l: sum(map(flatten,l),[]) if isinstance(l,list) else [l]
l = [2,[[1,2]],1]
flatten(l)

[2, 1, 2, 1]

In [1]:
### Finding Radius of gyration
from py4xs.hdf import h5xs,h5exp,lsh5,proc_d1merge
from py4xs.data2d import Data2d
from py4xs.slnxs import trans_mode
import numpy as np
import pylab as plt
import warnings,json

warnings.filterwarnings('ignore')

dt0  = h5xs("mica.h5", transField="em2_sum_all_mean_value")
dt  = h5xs("2048_B8.h5", transField="em2_sum_all_mean_value")

dt.load_d1s()
dt0.load_d1s()

dt.set_trans(transMode=trans_mode.external)
dt0.set_trans(transMode=trans_mode.external)

print(dt.d1s['2048_B8']['merged'][2080].trans)
print(dt0.d1s['mica']['merged'][10].trans)
#print(dt.d1s['2048_B8']['merged'][2080].overlaps)
#dt0.d1s['mica']['merged'][10].plot()

OSError: Unable to open file (unable to open file: name = 'mica.h5', errno = 2, error message = 'No such file or directory', flags = 1, o_flags = 2)

In [20]:
td = np.vstack((dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data))
qs = 0.007
qe = 10
rg = 15
# td = td[:, td[0, :] >= qs]     # find td for  greater than q=0.007
# td = td[:, td[0, :] <= qe]     # find td for smaller than q=1.3/1.5=0.086
# td[0, :] = td[0, :] * td[0, :]
# td[1, :] = np.log(td[1, :])
# rg, i0 = np.polyfit(td[0, :], td[1, :], 1)

In [21]:
from numpy.polynomial import polynomial as P
cnt = 0
while cnt < 10:
    if qe > 1.3/rg and 1.3/rg > qs+0.004: 
        qe = 1.3/rg
    td = np.vstack((dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data))
    td = td[:, td[0, :] >= qs]     # find td for  greater than q=0.007
    td = td[:, td[0, :] <= qe]     # find td for smaller than q=1.3/1.5=0.086
    td[0, :] = td[0, :] * td[0, :]
    td[1, :] = np.log(td[1, :])
    rg, i0 = np.polyfit(td[0, :], td[1, :], 1)
    i0 = np.exp(i0)
    
    print(cnt, qe, rg, i0)
    
    if rg<0:
        rg = np.sqrt(-rg * 3.)
    else:
        rg = 1e-6   # 
        print("likely strong inter-particle interaction ...")
        break
    cnt += 1
print(rg)

0 0.08666666666666667 -831.264016704801 20.377926211495332
1 0.026032341584365102 -6234.912831830037 281.9576616244667
2 0.026032341584365102 -6234.912831830037 281.9576616244667
3 0.026032341584365102 -6234.912831830037 281.9576616244667
4 0.026032341584365102 -6234.912831830037 281.9576616244667
5 0.026032341584365102 -6234.912831830037 281.9576616244667
6 0.026032341584365102 -6234.912831830037 281.9576616244667
7 0.026032341584365102 -6234.912831830037 281.9576616244667
8 0.026032341584365102 -6234.912831830037 281.9576616244667
9 0.026032341584365102 -6234.912831830037 281.9576616244667
136.76526786977064


In [72]:
%matplotlib widget
qmax = 5;
dmax=200.;

if dt.qgrid[-1] < qmax:
    qmax = dt.qgrid[-1]

tqgrid = np.arange(0, qmax, qmax / len(dt.qgrid))
tint = np.interp(tqgrid, dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data)
print(tint.shape)
tint[tqgrid * rg < 1.] = i0 * np.exp(-(tqgrid[tqgrid * rg < 1.] * rg) ** 2 / 3.)
#print(tint)

# tint -= tint[-10:].sum()/10
# Hanning window for reducing fringes in p(r)
tw = np.hanning(2 * len(tqgrid) + 1)[len(tqgrid):-1]

# plotting hanning window
tmp = np.hanning(2 * len(tqgrid) + 1)
#plt.plot(np.arange(0,len(tint[tqgrid * rg < 1.])), tint[tqgrid * rg < 1.])
#plt.yscale('log')
tint *= tw
#plt.show()

trgrid = np.arange(0, dmax, 1.)
kern = np.asmatrix([[rj ** 2 * np.sinc(qi * rj / np.pi) for rj in trgrid] for qi in tqgrid])
tt = np.asmatrix(tint * tqgrid ** 2).T
tpr = np.reshape(np.array((kern.T * tt).T), len(trgrid))
tpr /= tpr.sum()
print(tpr.shape)

(690,)
(200,)


In [39]:
tqgrid = np.arange(0, qmax, qmax / len(dt.qgrid))
tint = np.interp(tqgrid, dt.qgrid, dt.d1s['2048_B8']['merged'][2080].data)
tqgrid[0:10]

In [11]:
%matplotlib widget
file = '2302_EC-roi0.h5'
with h5py.File(file,'r') as hdf:
    xps = hdf.get(file.split('.h5')[0]  + '/xsp3_spectrum_array_data_monitor/data')
    xps = np.array(xps.get('xsp3_spectrum_array_data'))
print(xps.shape, xps.min(), xps.max())
plt.imshow(xps.reshape(-1, 4096))

(15495168,) 0.0 552.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [198]:
%matplotlib widget
### image registration
import cv2
from pystackreg import StackReg
import matplotlib.pyplot as plt


#load reference and "moved" image
ref = cv2.imread('unstained-2048_B8.png', cv2.IMREAD_GRAYSCALE)
Height, Width = ref.shape
mov = cv2.imread('tau-stained-2048_B8.png', cv2.IMREAD_GRAYSCALE)
mov = cv2.resize(mov, (Width,Height))

#Rigid Body transformation
sr = StackReg(StackReg.RIGID_BODY)
out_rot = sr.register_transform(ref, mov)
plt.imshow(out_rot, cmap='gray')

AttributeError: 'NoneType' object has no attribute 'shape'

In [206]:
%matplotlib widget

import glob
import tifffile
import numpy
import cv2
from pystackreg import StackReg
import matplotlib.pyplot as plt
os.chdir('/Users/bashit.a/Documents/Alzheimer/Codes')

with tifffile.TiffWriter('stack_images/my_image_stack.tif') as stack:
    for filename in glob.glob('stack_images/*.tif'): 
        stack.save(tifffile.imread(filename))

#load reference and "moved" image
ref = cv2.imread('stack_images/my_image_stack.tif')

#Rigid Body transformation
sr = StackReg(StackReg.RIGID_BODY)
out_previous = sr.register_transform_stack(ref, reference="first")
out_previous_int = out_previous.astype(numpy.int8)
                                           
tifffile.imsave('stack_images/my_aligned_stack.tif', out_previous_int)
plt.imshow(out_previous_int, cmap='gray')

<ipython-input-206-f25803115897>:13: DeprecationWarning: TiffWriter: data with shape (756, 1033, 3) and dtype 'int8'' are stored as RGB with contiguous samples. Future versions will store such data as MINISBLACK in separate pages by default unless the 'photometric' parameter is specified.
  stack.save(tifffile.imread(filename))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …